In [9]:
import pandas as pd
from pandas import json_normalize  # Import json_normalize for flattening JSON data

# Load the CSV file
file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"
df = pd.read_csv(file_path)  # Load the data from CSV into a DataFrame

# Define the columns containing key-value pairs stored as JSON-like strings
columns_with_pairs = ['new_car_detail']

# Function to process each column that contains key-value pairs
def process_column(column):
    # For each entry in the specified column, evaluate the string to a dictionary and normalize it
    flattened_dfs = df[column].apply(lambda x: pd.json_normalize(eval(x)) if pd.notnull(x) else pd.DataFrame())
    # Concatenate the resulting flattened DataFrames for all rows
    combined_df = pd.concat(flattened_dfs.tolist(), ignore_index=True)
    return combined_df

# Apply the function to each column in the list `columns_with_pairs`
result_dfs = [process_column(col) for col in columns_with_pairs]

# Concatenate all the processed DataFrames column-wise
expanded_df = pd.concat(result_dfs, axis=1)

# Combine the expanded columns with the `City` column from the original DataFrame
df_final_details = pd.concat([df[['City']], expanded_df], axis=1)

# Display the first few rows of the final DataFrame to verify the result
print(df_final_details.head(5))


<>:5: SyntaxWarning: invalid escape sequence '\.'
<>:5: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14120\3756714754.py:5: SyntaxWarning: invalid escape sequence '\.'
  file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"


        City  it      ft         bt        km transmission  ownerNo  \
0  Bangalore   0  Petrol  Hatchback  1,20,000       Manual        3   
1  Bangalore   0  Petrol        SUV    32,706       Manual        2   
2  Bangalore   0  Petrol  Hatchback    11,949       Manual        1   
3  Bangalore   0  Petrol      Sedan    17,794       Manual        1   
4  Bangalore   0  Diesel        SUV    60,000       Manual        1   

       owner      oem               model  modelYear  centralVariantId  \
0  3rd Owner   Maruti      Maruti Celerio       2015              3979   
1  2nd Owner     Ford       Ford Ecosport       2018              6087   
2  1st Owner     Tata          Tata Tiago       2018              2983   
3  1st Owner  Hyundai       Hyundai Xcent       2014              1867   
4  1st Owner   Maruti  Maruti SX4 S Cross       2015              4277   

                variantName        price priceActual priceSaving  \
0                       VXI     ₹ 4 Lakh                    

In [10]:
df_final_details.shape  # Get the dimensions of the DataFrame


(8369, 20)

In [11]:
import pandas as pd
import ast  # Import ast to safely evaluate strings with literal_eval
from pandas import json_normalize  # Import json_normalize for flattening JSON data

# Load the CSV file and read only the specific column
file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"
df = pd.read_csv(file_path, usecols=['new_car_overview'])

# Function to extract and normalize data from each row in 'new_car_overview'
def extract_data_from_row(row):
    cell_data = row['new_car_overview']
    if pd.notnull(cell_data):  # Check if the cell contains data
        try:
            # Safely evaluate the string as a dictionary
            data_dict = ast.literal_eval(cell_data)
            
            # Extract data from the 'top' section if available
            top_data = pd.DataFrame()
            if 'top' in data_dict:
                top_data = json_normalize(data_dict['top'])
            
            # Return the extracted data as a DataFrame
            return top_data
        except Exception as e:
            # Print error information if evaluation or extraction fails
            print(f"Error processing row: {row}")
            print(f"Exception: {e}")
            return pd.DataFrame()  # Return an empty DataFrame in case of an error
    return pd.DataFrame()  # Return an empty DataFrame if cell_data is null

# Process data in manageable chunks to optimize memory and CPU usage
chunk_size = 1000  # Set chunk size; adjust based on memory capacity
all_data = []  # List to hold processed chunks

# Loop through data in chunks for efficient processing
for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    chunk_df = df.iloc[start:end]  # Select a chunk of rows
    
    # Extract and normalize data for each row in the chunk
    chunk_data = [extract_data_from_row(row) for _, row in chunk_df.iterrows()]
    chunk_combined = pd.concat(chunk_data, ignore_index=True) if chunk_data else pd.DataFrame()
    
    # Add the combined chunk to the main list if not empty
    if not chunk_combined.empty:
        all_data.append(chunk_combined)

# Concatenate all processed chunks into a single DataFrame
df_combined = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Debugging: Print the shape and sample data of the combined DataFrame
print("Shape of df_combined:", df_combined.shape)
print(df_combined.head())

# Add a unique identifier column 'id' for each row to aid in pivoting
df_combined['id'] = df_combined.groupby('key').cumcount()

# Pivot the DataFrame to transform 'key' column values into separate columns
df_final_overview = df_combined.pivot_table(index='id', columns='key', values='value', aggfunc='first')

# Reset the index for a cleaner DataFrame without hierarchical indexing
df_final_overview = df_final_overview.reset_index(drop=True)

# Optionally, drop the 'id' column if it is no longer needed
df_final_overview = df_final_overview.drop(columns=['id'], errors='ignore')

# Debugging: Print the shape and sample data of the final pivoted DataFrame
print("Shape of df_final_overview:", df_final_overview.shape)
print(df_final_overview.head())


<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14120\885158902.py:6: SyntaxWarning: invalid escape sequence '\.'
  file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"


Shape of df_combined: (82682, 3)
                  key                  value  \
0   Registration Year                   2015   
1  Insurance Validity  Third Party insurance   
2           Fuel Type                 Petrol   
3               Seats                5 Seats   
4          Kms Driven           1,20,000 Kms   

                                                icon  
0  https://images10.gaadi.com/listing/vdp/co/v1/r...  
1  https://images10.gaadi.com/listing/vdp/co/v1/i...  
2  https://images10.gaadi.com/listing/vdp/co/v1/f...  
3  https://images10.gaadi.com/listing/vdp/co/v1/s...  
4  https://images10.gaadi.com/listing/vdp/co/v1/k...  
Shape of df_final_overview: (8369, 10)
key Engine Displacement Fuel Type     Insurance Validity    Kms Driven  \
0                998 cc    Petrol  Third Party insurance  1,20,000 Kms   
1               1497 cc    Petrol          Comprehensive    32,706 Kms   
2               1199 cc    Petrol          Comprehensive    11,949 Kms   
3            

In [12]:
import pandas as pd
import ast  # Import ast to safely evaluate string expressions
from pandas import json_normalize  # Import json_normalize to flatten nested JSON data

# Load the CSV file and read only the 'new_car_specs' column
file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"
df = pd.read_csv(file_path, usecols=['new_car_specs'])

# Add a unique identifier for each car to keep track of rows
df['car_id'] = df.index

# Function to extract and process data from each row in 'new_car_specs'
def extract_data_from_row(row):
    cell_data = row['new_car_specs']
    if pd.notnull(cell_data):  # Check if the cell has data
        try:
            # Safely evaluate the string as a dictionary
            data_dict = ast.literal_eval(cell_data)
            
            # Extract 'top' section if available
            top_data = pd.DataFrame()
            if 'top' in data_dict:
                top_data = json_normalize(data_dict['top'])
            
            # Extract 'list' section within nested dictionaries in 'data' section
            list_data = pd.DataFrame()
            if 'data' in data_dict:
                for data_entry in data_dict['data']:
                    if 'list' in data_entry:
                        # Flatten and concatenate 'list' section data
                        list_data = pd.concat([list_data, json_normalize(data_entry['list'])], ignore_index=True)
            
            # Combine 'top' and 'list' data into a single DataFrame
            combined_data = pd.concat([top_data, list_data], ignore_index=True) if not top_data.empty or not list_data.empty else pd.DataFrame()
            
            # Add 'car_id' to associate this data with the correct car
            combined_data['car_id'] = row['car_id']
            
            # Return the combined data for each car
            return combined_data
        except Exception as e:
            # Print an error message if processing fails
            print(f"Error processing row: {row}")
            print(f"Exception: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if there's an error
    return pd.DataFrame()  # Return an empty DataFrame if cell_data is null

# Process data in manageable chunks to optimize memory and CPU usage
chunk_size = 1000  # Set chunk size based on available memory
all_data = []  # List to store processed data chunks

# Loop over data in chunks for memory-efficient processing
for start in range(0, len(df), chunk_size):
    end = min(start + chunk_size, len(df))
    chunk_df = df.iloc[start:end]  # Select a chunk of rows
    
    # Extract and normalize data for each row in the chunk
    chunk_data = [extract_data_from_row(row) for _, row in chunk_df.iterrows()]
    chunk_combined = pd.concat(chunk_data, ignore_index=True) if chunk_data else pd.DataFrame()
    
    # Add the combined chunk to the main list if it’s not empty
    if not chunk_combined.empty:
        all_data.append(chunk_combined)

# Concatenate all processed chunks into a single DataFrame
df_combined = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Debugging: Print the shape and sample data of the combined DataFrame
print("Shape of df_combined:", df_combined.shape)
print(df_combined.head())

# Pivot the DataFrame to use 'car_id' as the index, creating separate columns for each key in 'value'
df_final_specs = df_combined.pivot_table(index='car_id', columns='key', values='value', aggfunc='first')

# Reset the index to clean up the DataFrame structure
df_final_specs = df_final_specs.reset_index(drop=True)

# Debugging: Print the shape and sample data of the final pivoted DataFrame
print("Shape of df_final_specs:", df_final_specs.shape)
print(df_final_specs.head())


<>:6: SyntaxWarning: invalid escape sequence '\.'
<>:6: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14120\20535502.py:6: SyntaxWarning: invalid escape sequence '\.'
  file_path = "D:\.intel\Ds\Capstone3\Project\concatenated_output.csv"


Shape of df_combined: (284200, 3)
         key      value  car_id
0    Mileage  23.1 kmpl       0
1     Engine     998 CC       0
2  Max Power   67.04bhp       0
3     Torque       90Nm       0
4      Seats          5       0
Shape of df_final_specs: (8369, 40)
key   Acceleration Alloy Wheel Size  BoreX Stroke Cargo Volumn   Color  \
0    15.05 Seconds              NaN    73 X 82 mm   235-litres   White   
1              NaN               16  79 X 76.5 mm   352-litres   White   
2     14.3 Seconds               14  77 X 85.8 mm   242-litres     Red   
3     14.2 Seconds               14           NaN   407-litres  Others   
4       12 Seconds               16  69.6 x 82 mm   353-litres    Gray   

key Compression Ratio Displacement Drive Type   Engine  \
0              11.0:1          998        FWD   998 CC   
1              11.0:1         1497        FWD  1497 CC   
2              10.8:1         1199        FWD  1199 CC   
3                 NaN         1197       FWD   1197 CC   
4  

In [13]:
# Concatenate the DataFrames df_final_details, df_final_overview, and df_final_specs horizontally
df_cars_combined = pd.concat([df_final_details, df_final_overview, df_final_specs], axis=1)

# Print the shape (number of rows and columns) of the combined DataFrame
print(df_cars_combined.shape)


(8369, 70)


In [14]:
# Display the last 10 rows of the combined DataFrame df_cars_combined
df_cars_combined.tail(10)


,City,it,ft,bt,km,transmission,ownerNo,owner,oem,model,...,Top Speed,Torque,Turbo Charger,Turning Radius,Tyre Type,Value Configuration,Values per Cylinder,Wheel Base,Wheel Size,Width
8359,Hyderabad,0,Diesel,MUV,"70,000",Manual,2,2nd Owner,Maruti,Maruti Ertiga,...,160 kmph,200Nm,Yes,5.2 metres,"Tubeless, Radial",DOHC,4,2740mm,NaN,1695mm
8360,Hyderabad,0,Petrol,Sedan,"16,925",Manual,1,1st Owner,Honda,Honda Amaze,...,NaN,110Nm,NaN,4.7,"Radial, Tubeless",SOHC,4,2470,NaN,1695
8361,Hyderabad,0,Petrol,Hatchback,"12,000",Manual,1,1st Owner,Maruti,Maruti Alto 800,...,NaN,69Nm,NaN,4.6,Tubeless Tyres,NaN,4,2360,NaN,1515
8362,Hyderabad,0,Diesel,SUV,"20,000",Automatic,1,1st Owner,BMW,BMW X3,...,213 kmph,400Nm,Yes,5.95 metres,"Tubeless,Radial",DOHC,4,2864mm,R19,1891mm
8363,Hyderabad,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Mahindra,Mahindra XUV500,...,175 kmph,330Nm,Yes,5.6 metres,"Tubeless,Radial",DOHC,4,2700mm,17,1890mm
8364,Hyderabad,0,Diesel,SUV,"60,000",Manual,1,1st Owner,Mahindra,Mahindra XUV500,...,175 kmph,330Nm,Yes,5.6 metres,"Tubeless,Radial",DOHC,4,2700mm,17,1890mm
8365,Hyderabad,0,Diesel,Sedan,"50,000",Automatic,1,1st Owner,Jaguar,Jaguar XF,...,250 Kmph,600Nm,Yes,5.75 metres,"Tubeless,Radial",DOHC,4,2909mm,20,1877mm
8366,Hyderabad,0,Diesel,Sedan,"1,20,000",Automatic,3,3rd Owner,BMW,BMW 5 Series,...,235kmph,400Nm,Twin,5.6,"Tubeless, Runflat",NaN,4,2975mm,R18,2126mm
8367,Hyderabad,0,Diesel,SUV,"50,000",Automatic,2,2nd Owner,Tata,Tata Nexon,...,154.19 kmph,260Nm,Yes,5.1m,Tubeless Radial Tyres,DOHC,4,2498mm,NaN,1811mm
8368,Hyderabad,0,Diesel,MUV,"1,00,000",Manual,1,1st Owner,Toyota,Toyota Innova,...,155 Kmph,200Nm,Yes,5.4 metres,"Tubeless,Radial",DOHC,4,2750mm,15,1765mm


In [15]:
# Save the combined DataFrame df_cars_combined to a CSV file without including the index column
df_cars_combined.to_csv(r'D:\.intel\Ds\Capstone3\Project\concatenated_cars.csv', index=False)


In [16]:
# Save the combined DataFrame df_cars_combined as a pickle file for efficient storage and retrieval
df_cars_combined.to_pickle('D:\.intel\Ds\Capstone3\Project\concatenated_cars.pkl')


<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Admin\AppData\Local\Temp\ipykernel_14120\4028318048.py:1: SyntaxWarning: invalid escape sequence '\.'
  df_cars_combined.to_pickle('D:\.intel\Ds\Capstone3\Project\concatenated_cars.pkl')
